In [1]:
from pathlib import Path
import pandas as pd
import torch
import hvplot.pandas

from diffhydro import TimeSeriesThDF
from diffhydro.pipelines import CalibrationRouter, CalibrationModule
from diffhydro.utils import SimpleTimeSeriesSampler

from utils.io import load_single_vpu

### New

In [3]:
plength_thr=10**4
node_thr=10**4
max_delay = 32
dt = 1/24

pred_len = 2048
device = "cuda:2"
vpu = "305"

root = Path("../data")

In [5]:
runoff, discharge, g = load_single_vpu(root, vpu, plength_thr=plength_thr, node_thr=node_thr)

#### Upstream stats computations ... ####


Computing breakpoints:   0%|          | 0/15562 [00:00<?, ?it/s]

#### Segmentation into subgraphs ... ####
Removing edges...


  0%|          | 0/15562 [00:00<?, ?it/s]

Segment graph into connected components....
Build subgraphs for each cluster and node-cluster map...


...

  0%|          | 0/19 [00:00<?, ?it/s]

Computing merged graphs node idxs...


  0%|          | 0/19 [00:00<?, ?it/s]

Match breakpoint nodes across clusters...


  0%|          | 0/19 [00:00<?, ?it/s]

In [6]:
tr_data = SimpleTimeSeriesSampler(runoff[:,:"1980"], discharge[:,:"1980"], 
                                  init_len=max_delay, 
                                  pred_len=pred_len)

te_data = SimpleTimeSeriesSampler(runoff[:,"1980":], discharge[:,"1980":], 
                                  init_len=max_delay, 
                                  pred_len=pred_len)

In [13]:
model = CalibrationRouter(g, max_delay, dt)
calib = CalibrationModule(model, tr_data, te_data).to(device)

In [14]:
results = calib.callibrate()

Training:   0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
(te_nse.hvplot(label="Test") *\
 tr_nse.hvplot(label="Train")).opts(ylim=(.999, 1))

:Overlay
   .Curve.Test  :Curve   [index]   (0)
   .Curve.Train :Curve   [index]   (0)

### Handling very large graphs

- For very large graphs, the GPU memory constraint may not allow to callibrate the full network parameters in one go.
- In this case, the CalibrationModule provides helpers to sequentially calibrate sub-basins from upstream to downstream.

In [ ]:
te_nse, tr_nse = calib.calibrate_staged_sequentially(n_epoch=10)